# Pismo Data Engineering Test

* [ ] Generate data on HDFS
* [ ] Validate data
* [ ] Read on Spark
* [ ] Statistics on initial data
* [ ] Clean up
* [ ] Apply rules
* [ ] Stats on Pismo data
* [ ] Save
* [ ] Validate 

# Architecture

This architecture was created to demonstrate a cluster and to simulate a cloud provider. 

A initial standalone application was made, but after thought and incentives, this was enriched to that format. 

The file directory...


# Generate Data

The data is always generated based on the script that I got from Github...

If data is already on HDFS, you can skip this cell. 



In [ ]:
# Generate data

from local_fake_data_generator import main as generate_fake_data

generate_fake_data()


In [5]:
import faker
import json

f = faker.Faker()

data = [ f.profile() for _ in range(10) ]

data


[{'job': 'Insurance risk surveyor',
  'company': 'Davis-Little',
  'ssn': '173-76-3430',
  'residence': '082 Lori Wall Suite 539\nSouth Robertborough, LA 23920',
  'current_location': (Decimal('35.656004'), Decimal('-6.569734')),
  'blood_group': 'B-',
  'website': ['https://www.sweeney-west.com/'],
  'username': 'robert60',
  'name': 'Steven Schaefer',
  'sex': 'M',
  'address': '7705 Robertson Grove Apt. 718\nWest Stephanie, DC 21895',
  'mail': 'kenneth39@gmail.com',
  'birthdate': datetime.date(1911, 2, 2)},
 {'job': 'Conservation officer, historic buildings',
  'company': 'Terrell, Powell and Garcia',
  'ssn': '467-25-6642',
  'residence': '7628 Valentine Curve Suite 891\nScottland, NY 58508',
  'current_location': (Decimal('33.830856'), Decimal('130.696313')),
  'blood_group': 'O-',
  'website': ['https://schmidt-leonard.org/'],
  'username': 'rowlandmatthew',
  'name': 'Ryan Hays',
  'sex': 'M',
  'address': '57729 James Squares Suite 537\nWhiteheadburgh, VI 81519',
  'mail': 'n

In [2]:
import pandas as pd

df = pd.DataFrame(data)
df

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Mining engineer,Guzman and Sons,864-99-0580,Unit 4621 Box 9045\nDPO AE 20367,"(65.1432955, 3.498644)",AB-,[http://www.mason-wright.biz/],roberttaylor,Krista Henry,F,"507 Rodriguez Alley\nNew Jefferyshire, TX 40387",heidiboyer@yahoo.com,1973-11-03
1,Arts development officer,York and Sons,683-72-8315,"37163 Lee Freeway Suite 124\nPort Joel, MN 11732","(-54.3438385, -165.518304)",O-,"[https://www.allen.info/, https://www.martinez...",davisbradley,Derrick Cook,M,"19148 Frey Islands\nMarkmouth, IL 18972",vanderson@yahoo.com,1969-12-07
2,Chartered accountant,Snyder Group,573-60-9692,"981 Chelsea Drive\nSouth Johnny, VI 57165","(66.8674585, 148.014779)",AB+,[http://www.francis-bright.com/],alejandra87,Nicole Brown,F,"86083 Ward Greens\nLake Brendan, PR 71516",edunn@gmail.com,1934-11-05
3,Health visitor,Vasquez-Thomas,500-72-0833,"46628 Michael Ferry Apt. 390\nWheelerstad, ME ...","(59.520707, 81.657198)",O-,"[https://perez.com/, http://www.rodriguez.biz/...",reneeruiz,Anthony White,M,"19451 Hardy Rapids\nClarkmouth, DE 13011",ginasanders@yahoo.com,1975-02-15
4,Media buyer,Reeves-Campbell,159-45-7514,6531 Howard Fords Apt. 757\nNorth Heathershire...,"(11.1238145, 66.299591)",B+,[https://dillon.info/],david03,Debra Harrington,F,"8913 Michael Drives\nNorth Phillipfurt, ME 32256",holly90@gmail.com,1985-02-18
5,Estate manager/land agent,"Padilla, Foster and Douglas",150-07-2495,"1031 Brandon Meadow\nWallfurt, WV 62358","(-81.1901965, 18.186904)",O-,"[https://www.sparks-shea.com/, http://james.in...",taylornancy,Ray Gonzalez,M,"585 Hale Brooks\nNorth Thomas, TX 40259",kerry79@yahoo.com,1991-12-09
6,"Engineer, structural",Krause-Wallace,299-84-9229,"42721 Villanueva Stream Apt. 649\nNew Eric, AS...","(80.714647, -127.688314)",A-,"[https://huynh.com/, http://anderson-burns.net/]",shawnsmith,Robert Rodriguez,M,"51721 Luna Skyway Apt. 904\nNew Johnburgh, NM ...",ewatkins@hotmail.com,1918-06-17
7,Minerals surveyor,Williams Ltd,489-15-8666,"8236 Christopher Drive Apt. 109\nWalkermouth, ...","(28.5064895, -154.535804)",O-,"[http://www.harris.org/, http://gonzalez.com/]",wilsonbrandon,James Lowe,M,"PSC 4703, Box 3500\nAPO AA 75786",zhoover@yahoo.com,1991-01-21
8,Geoscientist,Ramos-Evans,265-70-8837,"8652 Atkins Trail Apt. 500\nLisaland, DE 11189","(6.5117945, 111.822289)",O-,[http://foster-sanchez.com/],nancy29,Diana Flynn,F,"9241 Ray Harbor\nNew Jenniferburgh, TN 03153",danaroth@gmail.com,1946-12-04
9,Graphic designer,"Beck, Burns and Bond",482-32-4667,"8243 Rivas Ridge Suite 124\nWest Cindyhaven, N...","(-1.6905005, -22.974918)",AB-,"[http://www.douglas-hodge.com/, http://www.cla...",eware,Andrew Brown,M,"660 Duran Radial\nEast Catherine, AR 43976",meghanking@hotmail.com,1955-12-12


In [3]:
from pyspark.sql import SparkSession
from os import getenv

spark = SparkSession.builder.master(getenv('SPARK_MASTER')).getOrCreate()

spark_df = spark.createDataFrame(df)
spark_df

DataFrame[job: string, company: string, ssn: string, residence: string, current_location: struct<_1:decimal(38,18),_2:decimal(38,18)>, blood_group: string, website: array<string>, username: string, name: string, sex: string, address: string, mail: string, birthdate: date]

In [8]:
spark.__dict__

{'_sc': <SparkContext master=spark://spark-master:7077 appName=pyspark-shell>,
 '_jsc': JavaObject id=o13,
 '_jvm': <py4j.java_gateway.JVMView at 0x7fe4e9a87a90>,
 '_jsparkSession': JavaObject id=o24}

In [8]:
test = spark.read.option('header', True).csv('hdfs://hadoop-namenode:8020/sample-data/sales-data-sample.csv')
test.write.parquet('hdfs://hadoop-namenode:8020/sample-data/out/')

Py4JJavaError: An error occurred while calling o80.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=jovyan, access=WRITE, inode="/sample-data":root:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermissionWithContext(FSPermissionChecker.java:370)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:240)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1886)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp.mkdirs(FSDirMk

In [11]:
spark_df.write.parquet('hdfs://hadoop-namenode:8020/pismo-data/teste/')

Py4JJavaError: An error occurred while calling o71.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 14) (172.18.0.8 executor 0): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://hadoop-namenode:8020/pismo-data/teste.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckp

In [7]:
from hdfs import InsecureClient
client = InsecureClient('http://hadoop-namenode:9870', user='jovyan')

client.list('/pismo-data/')



[]

In [8]:
import json

with client.write('/pismo-data/test.json', encoding='utf-8') as writer:
    writer.write(json.dumps(data, default=str, indent=4))

HdfsError: Permission denied: user=jovyan, access=WRITE, inode="/":hadoop:supergroup:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:506)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:346)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:242)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1943)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1927)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1886)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.resolvePathForStartFile(FSDirWriteFileOp.java:323)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFileInt(FSNamesystem.java:2685)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.startFile(FSNamesystem.java:2625)
	at org

In [10]:
a = spark.read.option('multiline', True).json('hdfs://hadoop-namenode:8020/pismo-data/test.json')

a.write.parquet('hdfs://hadoop-namenode:8020/pismo-data/test-write/')

AnalysisException: [PATH_NOT_FOUND] Path does not exist: hdfs://hadoop-namenode:8020/pismo-data/test.json.